# Comparing Orbital Distributions


In [ ]:
using CairoMakie
using Arya
using LilGuys
using CSV, DataFrames

In [ ]:
using HDF5

In [ ]:
function read_traj(name)
    local positions, velocities, times
    
    h5open("$name/trajectory.hdf5", "r") do f
        positions = f["positions"][:, :, :]
        velocities = f["velocities"][:, :, :]
        times = -f["times"][:]
    end

    return positions, velocities, times
end

In [ ]:
function read_lmc_traj(name)
    
    # loads in trajectory of lmc in Vasiliev 2021
    lmc_file = "$name/lmc_traj.csv"
    lmc_traj = CSV.read(lmc_file, DataFrame)
    
    pos = reshape([lmc_traj.x lmc_traj.y lmc_traj.z]', (3, 1, :))
    vel = reshape([lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]', (3, 1, :))

    return pos, vel, -(lmc_traj.time)
end

In [ ]:
function subtract_traj(traja, trajb)
    if !isapprox(traja[3], trajb[3], atol=1e-3)
        println(extrema(traja[3]), " ", diff(traja[3])[1])
        println(extrema(trajb[3]), " ", diff(trajb[3])[1])
        error("trajectories do not match")
    end

    return traja[1] .- trajb[1], traja[2] .- trajb[2], traja[3]
end

In [ ]:
function read_distribution(name)
    return LilGuys.read_fits(joinpath(name, "peris_apos.fits"))
end

In [ ]:
plot_labels = Dict(
    "pericentre" => "pericentre / kpc",
    "peri_lmc" => "perilmc / kpc",
    "apocentre" => "apocentre / kpc",
    )

In [ ]:
function compare_peris(families; x="pericentre", legend_position=:lt)
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = plot_labels[x],
		ylabel = "pdf",
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df[:, x], normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend(position=legend_position)

	fig
end

In [ ]:
function compare_t_last_peris(families, legend_position=:lt)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "time since pericentre / Gyr",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.t_last_peri, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
function compare_apos(families)
    
	fig = Figure()
	ax = Axis(fig[1, 1],
		xlabel = "apocentre / kpc",
		ylabel = "pdf"
	)

    for (label, df) in families
    	bins, counts, err = LilGuys.histogram(df.apocentre, normalization=:pdf)
    	lines!(midpoints(bins), counts, label=label)
    end

    axislegend()

	fig
end

In [ ]:
using Printf

In [ ]:
using StatsBase

In [ ]:
function compare_stats(families)

    for (label, df) in families
        @printf "%16s%12.2f [%0.2f %0.2f] [[%0.2f %0.2f]]\n" label quantile(df.pericentre, [0.5, 0.16, 0.84, 0.0014, 0.9986])...
    end
end

# Initial scl comparisons

In [ ]:
families = [
    "strict" => read_distribution("reported_uncertanties"),
    "+gaia sys" => read_distribution("systematic_errors"),
    "+solar lsr sys" => read_distribution("sys_and_solar"),
    "all studies" => read_distribution("all_studies"),
    "iorio" => read_distribution("iorio")
    ]

In [ ]:
compare_stats(families)

In [ ]:
compare_peris(families)

In [ ]:
compare_apos(families)

## + trajectories

# LMC

## Trajectories

In [ ]:
families = [
    "EP20 (MW only)" => read_distribution("systematic_errors"),
    "V+21 (MW only)" => read_distribution("vasiliev_nolmc"),
    "V+21 (MW+LMC)" => read_distribution("vasiliev_lmc"),
    ]

In [ ]:
compare_stats(families)

In [ ]:
compare_peris(families)

In [ ]:
unique(df.t_last_peri)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel = "time since pericentre / Gyr",
    ylabel = "pericentre",
    limits=(0, nothing, 0, nothing)
)

for (label, df) in families
    scatter!(df.t_last_peri*T2GYR .+ 0.2*T2GYR * randn(length(df.t_last_peri)), df.pericentre, label=label => (; markersize=10), markersize=3, alpha=0.1)
end

df = families[end].second
scatter!(df.t_last_peri_lmc * T2GYR, df.peri_lmc, label="V+21 (LMC frame)" =>(; markersize=10), markersize=3, alpha=0.1)


axislegend(position=:lt)

fig

In [ ]:
families = [
    "L1.5" => read_distribution("vasiliev_lmc"),
    "L2M10" => read_distribution("vasiliev24_L2M10"),
    "L2M11" => read_distribution("vasiliev24_L2M11"),
    "L3M10" => read_distribution("vasiliev24_L3M10"),
    "L3M10rad" => read_distribution("vasiliev24_L3M10rad"),
    "L3M11" => read_distribution("vasiliev24_L3M11"),
    ];

In [ ]:
compare_peris(families, legend_position=:lt)

In [ ]:
compare_peris(families, x="peri_lmc", legend_position=:lt)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel = "time since pericentre / Gyr",
    ylabel = "pericentre",
    limits=(0, nothing, 0, nothing)
)

for (label, df) in families
    scatter!(df.t_last_peri*T2GYR .+ 0.2*T2GYR * randn(length(df.t_last_peri)), df.pericentre, label=label => (; markersize=10), markersize=3, alpha=0.1)
end

axislegend(position=:lt)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel = "time since pericentre / Gyr",
    ylabel = "pericentre",
    limits=(0, nothing, 0, nothing)
)

for (label, df) in families
    scatter!(df.t_last_peri_lmc*T2GYR .+ 0.2*T2GYR * randn(length(df.t_last_peri)), df.peri_lmc, label=label => (; markersize=10), markersize=3, alpha=0.1)
end

axislegend(position=:lt)

fig

# Compare trajectories

In [ ]:
traj = read_traj("vasiliev_lmc")

In [ ]:
traj_no = read_traj("vasiliev_nolmc")

In [ ]:
V_T2GYR = 0.97779

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])

lmc_x = LilGuys.lerp(lmc_traj.time, lmc_traj.x)
lmc_y = LilGuys.lerp(lmc_traj.time, lmc_traj.y)
lmc_z = LilGuys.lerp(lmc_traj.time, lmc_traj.z)
lmc_v_x = LilGuys.lerp(lmc_traj.time, lmc_traj.v_x)
lmc_v_y = LilGuys.lerp(lmc_traj.time, lmc_traj.v_y)
lmc_v_z = LilGuys.lerp(lmc_traj.time, lmc_traj.v_z)

times_v = traj[3] * T2GYR / V_T2GYR
pos = reshape([lmc_x.(times_v) lmc_y.(times_v) lmc_z.(times_v)]', (3, 1, :))
vel = reshape([lmc_v_x.(times_v) lmc_v_y.(times_v) lmc_v_z.(times_v)]', (3, 1, :))

In [ ]:
fig, ax = FigAxis(xlabel="y", ylabel="z")

scatter!(lmc_traj.y, lmc_traj.z)
scatter!(pos[2, 1, :], pos[3, 1, :])

fig

In [ ]:
traj_lmc = pos, vel, traj[3]
traj_scl_lmc = pos .- traj[1], vel .- traj[2], traj[3]


In [ ]:
function plot_r_t_traj!(traj; alpha=0.01, thin=1, color=:black, kwargs...)
    positions, velocities, times = traj
    for i in 1:thin:size(positions, 2)
        x = times * T2GYR
        y = calc_r(positions[:, i, :])
        lines!(x, y; rasterize=true, alpha=alpha, color=color, kwargs...)
    
    end
end

In [ ]:
function plot_x_y_traj!(traj; thin=1, x_direction=2, y_direction=3, alpha=0.01, color=:black, kwargs...)
    positions, velocities, times = traj
    for i in 1:thin:size(positions, 2)
        x = positions[x_direction, i, :]
        y = positions[y_direction, i, :]
        
        lines!(x, y; rasterize=true, alpha=alpha, color=color, kwargs...)
    
    end
end

In [ ]:
function compare_x_y_traj(trajectories; kwargs...)
    fig = Figure()
    ax = Axis(fig[1, 1], xlabel="y / kpc", ylabel="z / kpc",
        xgridvisible=false, ygridvisible=false, 
        aspect=DataAspect(),
    )
    
    for (i, (label, traj)) in enumerate(trajectories)
        plot_x_y_traj!(traj, label=label, color=COLORS[i]; kwargs...)
    end
        
    fig, ax
end

In [ ]:
function compare_r_t_traj(trajectories; kwargs...)
    fig = Figure()
    ax = Axis(fig[1, 1], xlabel="time / Gyr", ylabel = "Scl galactocentric distance / kpc",
        xgridvisible=false, ygridvisible=false
    )

    for (i, (label, traj)) in enumerate(trajectories)
        plot_r_t_traj!(traj, label=label, color=COLORS[i]; kwargs...)
    end
    
    fig, ax
end

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="time / Gyr", ylabel = "Scl galactocentric distance / kpc",
    xgridvisible=false, ygridvisible=false
)

plot_r_t_traj!(traj_no, label="MW only")
plot_r_t_traj!(traj, label="MW + LMC", color=COLORS[1])

axislegend(unique=true)
fig

In [ ]:
mkpath("figures")
figdir = "./figures/"

## 

In [ ]:
r2 = @. traj_scl_lmc[1][1, :, :]^2 + traj_scl_lmc[1][2, :, :]^2 + traj_scl_lmc[1][3, :, :]^2
r = sqrt.(r2)
idx_min = [argmin(row) for row in eachrow(r)]
t_scl_lmc_peri = traj_scl_lmc[3][idx_min]
r_scl_lmc_peri = [r[i, idx_min[i]] for i in 1:size(r, 1)]

In [ ]:
unique(idx_min)

In [ ]:
traj[3]

In [ ]:
traj[3][20:26]

In [ ]:
rb = LilGuys.calc_break_radius(9.4/V2KMS, 0.11 / T2GYR)

In [ ]:
LilGuys.kpc_to_arcmin(rb, 83.2)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="time of minimum / Gyr", ylabel="minimum distance (Scl-LMC) / kpc")

scatter!(t_scl_lmc_peri * T2GYR .+ 0.0003 * randn(length(t_scl_lmc_peri)), r_scl_lmc_peri)

fig

In [ ]:
V_T2GYR * 5/ T2GYR

In [ ]:
V_T2GYR * 0.25 / T2GYR

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="time / Gyr", ylabel = "radius / kpc",
    xgridvisible=false, ygridvisible=false
)

plot_r_t_traj!(traj, label="Scl-MW", color=COLORS[1])
plot_r_t_traj!(traj_scl_lmc, label="Scl-LMC", color=COLORS[2])
plot_r_t_traj!(traj_lmc, label="LMC-MW", color=:black, alpha=1, linewidth=3, )


axislegend(unique=true)

Makie.save("$figdir/scl_lmc_mw_r_t_samples.pdf", fig)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="y / kpc", ylabel="z / kpc",
    xgridvisible=false, ygridvisible=false, 
    aspect=DataAspect(),
)
plot_x_y_traj!(traj_scl_lmc, label="+LMC", color=:black)

fig

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="y / kpc", ylabel="z / kpc",
    xgridvisible=false, ygridvisible=false, 
    aspect=DataAspect(),
)
plot_x_y_traj!(traj_no, label="Scl, MW only")
plot_x_y_traj!(traj, label="Scl, MW + LMC", color=COLORS[1])
plot_x_y_traj!(traj_lmc, label="LMC", alpha=1, color=COLORS[2], linewidth=3)

axislegend(unique=true)

fig

### Other trajectories

In [ ]:
using OrderedCollections

In [ ]:
trajectories = OrderedDict(
    "L1.5" => read_traj("vasiliev_lmc"),
    "L2M10" => read_traj("vasiliev24_L2M10"),
    "L2M11" => read_traj("vasiliev24_L2M11"),    
    "L3M10" => read_traj("vasiliev24_L3M10"),
    "L3M10rad" => read_traj("vasiliev24_L3M10rad"),
    "L3M11" => read_traj("vasiliev24_L3M11"),
    )

In [ ]:
lmc_trajectories = OrderedDict(
    "L1.5" => read_lmc_traj("vasiliev_lmc"),
    "L2M10" => read_lmc_traj("vasiliev24_L2M10"),
    "L2M11" => read_lmc_traj("vasiliev24_L2M11"),    
    "L3M10" => read_lmc_traj("vasiliev24_L3M10"),
    "L3M10rad" => read_lmc_traj("vasiliev24_L3M10rad"),
    "L3M11" => read_lmc_traj("vasiliev24_L3M11"),
    )

In [ ]:
scl_lmc_trajectories = OrderedDict(key => subtract_traj(trajectories[key], lmc_trajectories[key]) for key in keys(trajectories))

In [ ]:
fig, ax = compare_x_y_traj(trajectories, thin=10, alpha=0.03)

Legend(fig[1,2], ax, unique=true)
fig

In [ ]:
fig, ax = compare_x_y_traj(trajectories, x_direction=1, thin=10, alpha=0.03)

ax.xlabel = "x / kpc"
Legend(fig[1,2], ax, unique=true)
fig

In [ ]:
fig, ax = compare_x_y_traj(lmc_trajectories, alpha=1)
Legend(fig[1,2], ax, unique=true)
fig

In [ ]:
fig, ax = compare_x_y_traj(scl_lmc_trajectories, thin=10, alpha=0.03)

Legend(fig[1,2], ax, unique=true)
fig

In [ ]:
fig, ax = compare_r_t_traj(trajectories, thin=10, alpha=0.02)

Legend(fig[1,2], ax, unique=true)
fig

In [ ]:
fig, ax = compare_r_t_traj(lmc_trajectories, alpha=1)
ax.ylabel = "LMC - MW distance / kpc"
Legend(fig[1,2], ax, unique=true)
fig

In [ ]:
fig, ax = compare_r_t_traj(scl_lmc_trajectories, thin=10, alpha=0.02)

ax.ylabel = "Scl - LMC distance / kpc"
Legend(fig[1,2], ax, unique=true)
fig